In [1]:
!pip install -q transformers datasets matplotlib
!!pip install -q accelerate torchinfo huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


["\x1bWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv\x1b\x1b",
 '\x1b',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b A new release of pip is available: \x1b\x1b23.0.1\x1b\x1b -> \x1b\x1b23.1.2\x1b',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b To update, run: \x1b\x1bpython -m pip install --upgrade pip\x1b']

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.3.0).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter
import torch 
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast
from tqdm.notebook import tqdm
from transformers import T5ForConditionalGeneration, T5Config, AdamW
from tqdm.auto import tqdm
from accelerate import Accelerator

In [5]:
file_id = '1yXvDTRRxjCLyAf4icU1sEMKF0NpssCnC'
destination = 'human_proteins.tsv'
hf_name = "t5-small"
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

In [6]:
file_id = '1yXvDTRRxjCLyAf4icU1sEMKF0NpssCnC'
destination = 'human_proteins.tsv'

def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")


download_file_from_google_drive(file_id, destination)

0it [00:00, ?it/s]



saved all the data to human_proteins.tsv. total size : 12.9 MB


In [7]:
download_file_from_google_drive("1FLrC9kK5-R_NwjmX_WTqT8YqchzS9zIN", "all_proteins.tsv")

0it [00:00, ?it/s]



saved all the data to all_proteins.tsv. total size : 294.963 MB


In [8]:
dataframe = pd.read_csv("human_proteins.tsv", sep="\t")
dataframe = dataframe[dataframe["Sequence"].str.len() <= 510]
dataframe = pd.read_csv("human_proteins.tsv", sep="\t")
dataframe = dataframe[dataframe["Sequence"].str.len() <= 510]
dataframe["Protein Name"] = [v.split("(")[0] for v in dataframe["Protein Name"]]
dataframe["Sequence"] = [" ".join(v) for v in dataframe["Sequence"]]

with open("Sequences.txt", "w") as f:
    f.write("\n".join(dataframe["Sequence"].values))
    
dataframe

,Sequence,Protein Name
0,M R W Q E M G Y I F Y P R K L R,Mitochondrial-derived peptide MOTS-c
4,M P G W F K K A W Y G L A S L L S F S S F I L ...,Clarin-2
6,M K R R Q K R K H L E N E E S Q E T A E K G G ...,Protein FAM170A
7,M D D A D P E E R N Y D N M L K M L S D L N K ...,Synaptonemal complex central element protein 3
8,M A F S D L T S R T V H L Y D N W I K D A D P ...,Elongation of very long chain fatty acids prot...
...,...,...
20417,M P L A S P I Q H H E V T R G V A P S M A L R ...,Putative uncharacterized protein PRO3102
20418,M V R P H L L K K K I L G R V W W L M P V V L ...,Putative uncharacterized protein PRO2829
20419,M A E T Y R R S R Q H E Q L P G Q R H M D L L ...,Putative uncharacterized protein DKFZp434L187
20420,M A H H S L N T F Y I W H N N V L H T H L V F ...,Putative uncharacterized protein encoded by LI...


In [9]:
dataframe2 = pd.read_csv("all_proteins.tsv", sep="\t")
dataframe2 = dataframe2[dataframe2["sequence"].str.len() <= 510]
dataframe2 = dataframe2[dataframe2["sequence"].str.len() <= 510]
dataframe2["protein_name"] = [v.split("(")[0] for v in dataframe2["protein_name"]]

with open("names.txt", "w") as f:
    f.write("\n".join([v.split("(")[0].strip() for v in dataframe2["protein_name"].values]))  

dataframe2

,Unnamed: 0,sequence,protein_name,length
0,0,MRWQEMGYIFYPRKLR,Mitochondrial-derived peptide MOTS-c,16
1,1,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,Clarin-2,232
2,2,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,Protein FAM170A,330
3,3,MDDADPEERNYDNMLKMLSDLNKDLEKLLEEMEKISVQATWMAYDM...,Synaptonemal complex central element protein 3,88
4,4,MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...,Elongation of very long chain fatty acids prot...,281
...,...,...,...,...
619007,529106,MNSNAPAAVIVLAAGAGTRMKSKLPKVLHEIGGRSLLMHAITAARG...,Bifunctional protein GlmU [Includes: UDP-N-ace...,492
619008,529107,MSATGSDPSRRPVDLPDLSREAVPGEKVALAPGQLQLRPTRRGKAP...,Probable dual-specificity RNA methyltransferas...,428
619009,529108,MVLASHNAKKLRELQRILAPAVPGLEAEQIVSAAGLGLPDVVEDAV...,dITP/XTP pyrophosphatase,203
619010,529109,MLPVLTADALRTAEQAHWDEHPGDDLMGRAAAEVARHATEMLGDGP...,ADP-dependent,492


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mlewand/PROT5-small", max_length=512,
                                                   padding=True)

In [11]:
ids = tokenizer.encode("M R W Q E M G Y I F Y P R K L R")
print(ids, tokenizer.decode(ids))
ids = tokenizer.encode("Protein FAM170A")
print(ids, tokenizer.decode(ids))

[1, 118, 89, 204, 116, 100, 118, 92, 131, 110, 115, 131, 88, 89, 104, 85, 89, 2] <s> M R W Q E M G Y I F Y P R K L R</s>
[1, 249, 9701, 12, 25, 2] <s> Protein FAM170A</s>


In [12]:
_tokenized_X = []
_tokenized_Y = []

tokenized_X = []
tokenized_Y = []
sizes_X = []
sizes_Y = []

max_length_inp = 512
max_length_out = 32 

for x in tqdm(dataframe["Sequence"]):
    input_tokens = tokenizer.encode(x)
    tokenized_X.append(input_tokens)
    sizes_X.append(len(input_tokens))

for y in tqdm(dataframe["Protein Name"]):
    out_tokens = tokenizer.encode(y)
    tokenized_Y.append(out_tokens)
    sizes_Y.append(len(out_tokens))
    
for x,y in tqdm(zip(_tokenized_X, _tokenized_Y)):
   if len(x) <= max_length_inp and len(y) <= max_length_out:
    tokenized_X.append(x)
    tokenized_Y.append(y)

  0%|          | 0/12617 [00:00<?, ?it/s]

  0%|          | 0/12617 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [13]:
train_val_split = 0.9
l = len(tokenized_X)
split_idx = int(train_val_split * l)

idxes = np.arange(l)
r = np.random.permutation(idxes)

train_idxs = r[:split_idx]
val_idxs = r[split_idx:]

train_X, train_Y = [], []
val_X, val_Y = [], []


for idx in train_idxs:
    train_X.append(tokenized_X[idx])
    train_Y.append(tokenized_Y[idx])
    
for idx in val_idxs:
    val_X.append(tokenized_X[idx])
    val_Y.append(tokenized_Y[idx])   
    
print(f"train sizes ;  X : {len(train_X)}, Y : {len(train_Y)}")
print(f"val sizes ;  X : {len(val_X)}, Y : {len(val_Y)}")


train sizes ;  X : 11355, Y : 11355
val sizes ;  X : 1262, Y : 1262


In [16]:
class CustomAADataset(Dataset):
  def __init__(self, X, Y, max_length=512, max_length2=32):
    self.max_length = max_length
    self.max_length2 = max_length2 
    self.X = X
    self.Y = Y
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    input_tokens = self.X[idx]
    output_tokens = self.Y[idx]

    # Pad input and output tokens
    input_tokens = input_tokens + [0] * (self.max_length - len(input_tokens))

    output_tokens = output_tokens + [0] * (self.max_length2 - len(output_tokens))

    return {"input_ids": torch.tensor(input_tokens, dtype=torch.long),
            "labels": torch.tensor(output_tokens, dtype=torch.long),
            }


In [17]:
# Load data
from datasets import Dataset as Dataset_hf

dataset = Dataset_hf.from_dict({"source": dataframe["Sequence"].values, "target": dataframe["Protein Name"].values})

In [18]:
train_dataset = CustomAADataset(train_X, train_Y, max_length_inp, max_length_out)
val_dataset = CustomAADataset(val_X, val_Y, max_length_inp, max_length_out)

train_dataset[0]

{'input_ids': tensor([  1, 118,  99,  85,  88,  89,  87, 100,  89,  88,  89,  90, 109,  88,
         116,  89,  90,  85,  89, 108,  90, 108,  92, 100, 108,  92, 104, 110,
         108, 101,  85,  92, 100, 100, 100, 108, 100, 108, 100, 101, 100, 108,
         100, 100, 100, 100,  87,  89, 116, 116, 115,  85, 100, 116,  90,  85,
         116,  88,  92,  85, 116, 101,  87,  89, 204,  92,  92, 101,  87,  85,
          88,  89, 100, 129, 110, 100,  92,  92,  92,  92,  88,  90, 108,  88,
          90, 100, 115,  92, 109, 104, 115,  89,  87,  88,  88,  89,  90,  87,
          87,  87,  90, 100, 108,  87,  89, 116,  88,  87, 104,  88,  88, 131,
          90, 131, 110,  87,  85, 110, 109, 118,  87, 110,  85, 116,  99,  88,
         129, 104,  89,  85, 109,  85,  90,  92, 110, 111,  87, 115, 110,  90,
          92,  89, 115,  88, 131, 131,  89,  89, 104, 115,  88,  87, 204, 116,
          99,  90, 110,  89, 129,  99,  85,  90,  85,  99, 108, 111, 115, 101,
         104, 110,  88,  89, 100,  88, 

In [19]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=None, padding=True, max_length=512)

# Load T5 model
config = T5Config.from_pretrained("mlewand/PROT5-small")
model = T5ForConditionalGeneration(config)

In [73]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=None, padding=True, max_length=512)

# Load T5 model
#config = T5Config.from_pretrained("mlewand/PROT5-small")
#model = T5ForConditionalGeneration(config)
model_name = "t5-small"
config = T5Config.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True)

In [74]:
dats = None
eval_d = None

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    global dats
    pred_ids = torch.argmax(logits[0],dim=-1)
    return pred_ids

def compute_metrics(pred):
    global eval_d
    eval_d = pred
    labels = pred.label_ids
    # labels = np.argmax(labels, axis=1)
    preds = pred.predictions

    tot = np.sum(labels != 0) + 1e-10
    classified_correctly = 0
    exact_matches = 0
    for i in range(preds.shape[0]):
        mask = preds[i, :] != 0
        classified_correctly += np.sum(preds[i, mask] == labels[i, mask])
        exact_matches += np.all(preds[i,:] == labels[i, :])
            
    return {"matches": classified_correctly / tot, "exact_matches" : exact_matches / preds.shape[0]}

In [75]:
training_args = Seq2SeqTrainingArguments(
    output_dir="PROT5-small",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    num_train_epochs=50,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    fp16=True,  # Enable mixed precision training if your GPU supports it
    push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=None,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics= preprocess_logits_for_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Matches,Exact Matches
1,1.385100,1.210753,0.389653,0.000000
2,1.159100,1.115095,0.437474,0.000000
3,1.087000,1.107715,0.409226,0.000000
4,1.034000,1.035388,0.444985,0.000000
5,0.992300,1.007090,0.444774,0.000000
6,0.955400,0.966385,0.471646,0.000792
7,0.922400,0.935863,0.492488,0.000000
8,0.896100,0.917027,0.500212,0.000792
9,0.871300,0.902732,0.509733,0.000792
10,0.851500,0.887842,0.508570,0.000000


TrainOutput(global_step=35500, training_loss=0.7375330303353324, metrics={'train_runtime': 2994.8567, 'train_samples_per_second': 189.575, 'train_steps_per_second': 11.854, 'total_flos': 7.6840307785728e+16, 'train_loss': 0.7375330303353324, 'epoch': 50.0})

In [ ]:
model.push_to_hub("mlewand/PROT5-small-correct-eval")

In [39]:
labels = eval_d.label_ids
    # labels = np.argmax(labels, axis=1)
preds = eval_d.predictions

In [58]:
k = 0
for a,b in zip(labels,preds):
    mask = a != 0
    k += np.sum(a[mask] ==b[mask])
k / np.sum(labels != 0) 

0.00017333597464342314

In [59]:
print(k)

7
